### Global

#### Librairies

In [26]:
import os
import json
import numpy as np
import torch
import torchaudio
from torch.utils.data import TensorDataset, DataLoader

#from google.colab import drive
from torch import nn
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import LabelEncoder

#### Variables

In [27]:
#path_drive = "/content/drive/MyDrive/projet_signal/"
path_drive = "./"
path_data = path_drive + "data/"
path_music = path_data + "music/"

themes = ["classic", "techno", "rap", 'metal']

#### Lecture des ressources

In [28]:
# drive.mount('/content/drive', force_remount=True)
# os.listdir(path_data)

#### Fonctions utilitaires

In [29]:
def throw_error(text):
  print("Error: " + text)

def throw_info(text):
  print("Info: " + text)

### Dataset

#### Librairies

#### Variables

In [30]:
youtube_classic = [
    "https://youtu.be/k1-TrAvp_xs?si=r-Jieh8TcaTeNKDZ",
    "https://youtu.be/mmCnQDUSO4I?si=LyCQ3MV66-WhBr9e",
    "https://youtu.be/eab_eFtTKFs?si=WnQk6pAHhxHv59wG",
    "https://youtu.be/1UhOH-5ojhY?si=arn71qGOCYIT4TGk",
    "https://youtu.be/l4i51b8ZS-4?si=zg-E0JoO6BEXXH9g",
    "https://youtu.be/tWTJcUj1n98?si=OusPXXPwVorTxKx0",
    "https://youtu.be/SQTp3jL8OJw?si=m92DJ5P9FNazgW03",
    "https://youtu.be/2y09pD1r-Qs?si=LTVrViG7xMFf3KLU",
    "https://youtu.be/8op3Efsfuyk?si=K7gtC7FFLIkOjAET",
    "https://youtu.be/L25qDEvVQts?si=e827obk5HjfENccv",
    "https://youtu.be/t894eGoymio?si=yRk3KvFoa4wILRFX",
    "https://youtu.be/reI43yUCaUI?si=H-5ILNZl5CaPEx_a",
    "https://youtu.be/CRhnXjlqfrw?si=ALQurYsu9QodAdhD",
    "https://youtu.be/3fYJvcBSNgU?si=m3BPHYY8LOI_CLOv",
    "https://youtu.be/muVoIHKlaoE?si=caztQNpNkHi6BmoQ",
    "https://youtu.be/BlMuh46M-L0?si=6TvQC51REFNkG76X",
    "https://youtu.be/Tv40mcAM1ZA?si=7PiiYLMUp_9fpLrR",
    "https://youtu.be/O9nrehvbHrs?si=s5zB7HQa3aZ6PEwq",
]

youtube_techno = [
    "https://youtu.be/I7xgCtn8374?si=Rjb52k30J6tLnFzr",
    "https://youtu.be/kivuDS-6HbQ?si=NoPZAdQ3Y0vqBk9s",
    "https://youtu.be/capJ9NLuW9E?si=GoCsnNWc8WXNlK50",
    "https://youtu.be/capJ9NLuW9E?si=FkMzJCZM8UiTV7IB",
    "https://youtu.be/Lgs9QUtWc3M?si=U4nq-0-l2kFT1GOo",
    "https://youtu.be/X90VQ_BSr1Y?si=rj1QyHkO0s6EGsL1",
    "https://youtu.be/X90VQ_BSr1Y?si=JVIXB-640tHkDTF1",
    "https://youtu.be/3x2ABSAMVno?si=B3Wkd6FeZ9IusQB3",
    "https://youtu.be/1-xGerv5FOk?si=N9qbWYkBlnvGscmV",
    "https://youtu.be/6FNHe3kf8_s?si=Y3iUrnIrMcAzG1aM",
    "https://youtu.be/f4gODZFkTJg?si=Y6KvqKZGbRZFJ7X7",
    "https://youtu.be/43p-kZtgVSE?si=nBiXiOXKjFASHtpc",
    "https://youtu.be/HaywYXQHgRA?si=BzK9Sef3vQ3xg5t-",
    "https://youtu.be/mNrzmpA8JU4?si=2cQw_dqUQ3HRCyZI",
    "https://youtu.be/upQe8EeSyZU?si=VaBEWTi3GJR7mmxT",
    "https://youtu.be/Sr1bLLvsbh0?si=eKc0hzPVWhMYrdl1",
    "https://youtu.be/LOkTUFaxpkc?si=pBTukZKzv0x69o8v",
    "https://youtu.be/mahibKo1qVc?si=H31Wq_Pbo0cQqYgW",
    "https://youtu.be/HegA9-k0H_I?si=iD5Kkbs2Ihj3KKXC",
    "https://youtu.be/ayo-ckzXWJE?si=r-50HHFxyOJRVRf3",
    "https://youtu.be/fc7A6XIF2t8?si=hfAenJaQLLdCj7Zs",
    "https://youtu.be/bID5Srt_YDA?si=egoaVJ9h-zocDoWS",
    "https://youtu.be/wMYQ5x52CnA?si=4ejbtdqXNi1dlonI",
    "https://youtu.be/wGM2nf4ay0w?si=riumjaxKTdbU8wHS",
    "https://youtu.be/6-nWCiSgvcE?si=oFI1kN0CqdR6m2Vk",
    "https://youtu.be/ALZHF5UqnU4?si=ob2nuatGBlDcRmdG",
    "https://youtu.be/ko-6yzRpX60?si=5YCY9Uva71CN_Z8I",
]

youtube_rap = [
    "https://youtu.be/zJPEOhz5zZ4?si=qoWc8Hw6Xo47-x7B",
    "https://youtu.be/WcX37NouKxw?si=Yi_P63GHH4ZGZiGe",
    "https://youtu.be/NOspdXm3eRY?si=DNlSRLS85tb-4ctk",
    "https://youtu.be/No3Cr6ph-9U?si=n0v74kOSR6qGJapg",
    "https://youtu.be/X8yNru9psQE?si=wy0jYFdjO-QDFHuw",
    "https://youtu.be/yuzCPztZU7I?si=guYJ3Cnqc_iT_Fg9",
    "https://youtu.be/qOnJwDYzzsM?si=DXf14MPPstcaowdE",
    "https://youtu.be/HF2q24RkrH4?si=rMztSICDs7XkcNa0",
    "https://youtu.be/mVUKyjg1EJA?si=AvoRi5sCF0T_J_5C",
    "https://youtu.be/DU4NtqyTnuM?si=9E1OIMTMwP34Ng_j",
    "https://youtu.be/Uoo9P7vPUs8?si=iFnAO24mUgTRIZb2",
    "https://youtu.be/Qq7Q6CLCfvs?si=jd9TzAnWgkXW6fKd",
    "https://youtu.be/pxjsZK_fkO4?si=lNbGdqUCLLn9avJB",
    "https://youtu.be/_La5RaE5GF4?si=o20tN4iZARpXkp0T",
    "https://youtu.be/xFRIddDw8bw?si=zBRvOmGhGMJYb3-M",
    "https://youtu.be/2A0FHgB1Bw0?si=c8qiEBkGfgMZ_L03",
    "https://youtu.be/2A0FHgB1Bw0?si=NFsM9fDKk0hG6RE7",
    "https://youtu.be/IJBpPAcdeE8?si=psXdISI8bIeHmzGU",
    "https://youtu.be/Hd3J5Djfek4?si=HvsdmAxjbqAbX-d8",
    "https://youtu.be/pN-TLYBDkWc?si=emjGAk74Fhiik3jG",
    "https://youtu.be/xUb0eZbfjj0?si=jMBa0MhtzDCJJ11r",
    "https://youtu.be/9Oh3knWDBnA?si=FOTpips7ANuJCeQD",
    "https://youtu.be/9Oh3knWDBnA?si=3xK2OuIQuyiopm43",
    "https://youtu.be/Te84vYEMf-4?si=VOxsNXM4WV_vMAKY",
    "https://youtu.be/__nCSbPDU2s?si=d8_ypD6aTxLi9Fw-",
    "https://youtu.be/-mOoTp8ZC8o?si=NqxfNOI1Y4yW3JAd",
    "https://youtu.be/1esyx2sHsuk?si=Fi6QoBZkW-TJPfyu",
    "https://youtu.be/LJaCSPxDrSg?si=H4wl8mhkGFvsYo2O",
    "https://youtu.be/VfzhSbFPSRg?si=pp8v9jAqMAXg1E22",
    "https://youtu.be/iXzBdNjpYw0?si=tNNuRSgeUzdovodz",
    "https://youtu.be/507d9xto6Og?si=oJfPA_PPkZgt9biY",
    "https://youtu.be/507d9xto6Og?si=4hrJhoisonz9xeNV",
    "https://youtu.be/i3mIs4TQZtY?si=0QAallKg_QOpQ5dB",
    "https://youtu.be/A-fnvTdjk8k?si=6zGLjAYa0tfbbta7",
    "https://youtu.be/yZKfCQ4pqiE?si=mqmeoWW0o8FVwmJ1",
    # "https://youtu.be/M1uedhyydjE?si=hGoG0s63evv0mit8",
    # "https://youtu.be/iIHdTMHWAic?si=k3mza2Z3317hj9mj",
    # "https://youtu.be/S3LgSXwvHo0?si=G-W9XPYHQgwRzIo2",
    # "https://youtu.be/Do95DqTI88g?si=KfP4qyAgrmcvW4D4",
    # "https://youtu.be/QoPTeC8JvVo?si=xT8POgtypzV2-DL1",
    # "https://youtu.be/QoPTeC8JvVo?si=l6R64wTufq0V9Csa",
    # "https://youtu.be/85d4uAx9nvs?si=Zy8D1NIg_m9fZ8SM",
]

youtube_metal =  [
    'https://www.youtube.com/watch?v=CD-E-LDc384&ab_channel=Metallica',
    'https://www.youtube.com/watch?v=CfPKxm9O04Y&ab_channel=TheGrande',
    'https://www.youtube.com/watch?v=TmsnpuNOF8o&ab_channel=Slayer-Topic',
    'https://www.youtube.com/watch?v=1kbon057vPk&ab_channel=UnionSquareMusic',
    'https://www.youtube.com/watch?v=-0Ao4t_fe0I&ab_channel=GhostBCVEVO',
    'https://www.youtube.com/watch?v=LplPi2CxNHI&ab_channel=OzzyOsbourne',
    'https://www.youtube.com/watch?v=hEihQoMJC98&ab_channel=Megadeth-Topic',
    'https://www.youtube.com/watch?v=xGC2iP5ss6E&ab_channel=RoadrunnerRecords',
    'https://www.youtube.com/watch?v=1w7OgIMMRc4&ab_channel=GunsNRosesVEVO',
    'https://www.youtube.com/watch?v=yMVV_HsHcX0&ab_channel=JudasPriestVEVO',
    'https://www.youtube.com/watch?v=l_bJVVpSi7k&ab_channel=ThePowerMetalChannel',
    'https://www.youtube.com/watch?v=0UIB9Y4OFPs&ab_channel=DEFLEPPARD',
    'https://www.youtube.com/watch?v=0qanF-91aJo&ab_channel=BlackSabbath',
    'https://www.youtube.com/watch?v=Pn0w7vciQCE&ab_channel=Airbourne',
    'https://www.youtube.com/watch?v=_NdHySPJr8I&ab_channel=acdcVEVO',
    'https://www.youtube.com/watch?v=uSquiIVLhrQ&ab_channel=80sclassicrocksongs'
    'https://www.youtube.com/watch?v=2lvs2FzF64o&ab_channel=RHINO'
    'https://www.youtube.com/watch?v=AkFqg5wAuFk&ab_channel=Pantera'
    'https://www.youtube.com/watch?v=hFjoOOPU6Xo&ab_channel=Slayer-Topic',
    'https://www.youtube.com/watch?v=uGHsxMqpL0c&ab_channel=AnthraxVEVO'
]

#### Téléchargement des musiques

In [31]:
#!pip install pytube
from pytube import YouTube

data = []

def get_youtube_music(videos, theme):
  for i, url in enumerate(videos, start=1):
    try:
      yt = YouTube(url)
      title = theme + "_" + str(i) + ".mp3"
      yt.streams.filter(only_audio=True).first().download(path_music + theme, filename=title)

      throw_info(f"video {url} downloaded!")

      data.append({"music": title, "theme": theme})
    except Exception as e:
      throw_error(f"can't download video {url}!\n{e}")

# get_youtube_music(youtube_classic, themes[0])
# get_youtube_music(youtube_techno, themes[1])
# get_youtube_music(youtube_rap, themes[2])
# get_youtube_music(youtube_metal, themes[3])

In [32]:
#training params
learningRate = 0.0005
batchSize = 250
epochs = 200
fft_length = 10000
dataset_length = 2000
device = 'cuda' if torch.cuda.is_available() else 'cpu'

### Chargement du dataset

In [33]:
fourier_transforms = []
labels = []

for i in range(0, dataset_length):
  randIndex = torch.randint(0, len(themes), (1,))[0]
  theme = themes[randIndex]
  files_path = "./data/" + theme
  musics = os.listdir(files_path)
  files_path = files_path + '/'
  music = musics[torch.randint(0, len(musics), (1,))[0]]
  throw_info(f"processing {music} fft...")

  waveform, sample_rate = torchaudio.load(files_path + music)
  #convert into mono
  waveform = 0.5 * (waveform[0] + waveform[1])
  randIndex = torch.randint(0, waveform.shape[0] - fft_length, (1,))[0]
  fft = torch.fft.fft(waveform[randIndex : randIndex + fft_length])

  fourier_transforms.append(fft)
  labels.append(themes.index(theme))

# Convert lists to tensors
labels = torch.tensor(labels)

Info: processing Johannes Brahms - Danse hongroise N°5.mp3 fft...
Info: processing Gringe - Qui dit mieux (feat. OrelSan, Vald, Suikon Blaz AD) [ENFANT LUNE].mp3 fft...
Info: processing B.B. Jacques - PÉTALESft. Oxmo Puccino.mp3 fft...
Info: processing Marshmello - Alone (Official Music Video).mp3 fft...
Info: processing OrelSan - Rêves bizarres (feat. Damso) [CLIP OFFICIEL].mp3 fft...
Info: processing B.B. Jacques - PÉTALESft. Oxmo Puccino.mp3 fft...
Info: processing B.B. Jacques - PÉTALESft. Oxmo Puccino.mp3 fft...
Info: processing Johann Strauss Sr. - Radetzky March.mp3 fft...
Info: processing Judas Priest - Electric Eye (Official Video).mp3 fft...
Info: processing Johannes Brahms - Danse hongroise N°5.mp3 fft...
Info: processing Creeds - Push Up (Lyrics)  Tiktok.mp3 fft...
Info: processing Tourner Dans Le Vide (REQVIEM Techno Remix).mp3 fft...
Info: processing Le lac des cygnes ( Tchaikovski ).wmv.mp3 fft...
Info: processing King Diamond - The Family Ghost [OFFICIAL VIDEO].mp3 fft.

### Entraînement perceptron multicouche

In [34]:
import torch
import torch.nn.functional as F

class MusicGenreCNN(nn.Module):
    def __init__(self):
        super(MusicGenreCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=10)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv1d(in_channels=128, out_channels=1, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2, padding=0)
        #self.fc1 = nn.Linear(623, 256)  # Adjust the input size based on your music input size
        self.fc1 = nn.Linear(1248, 256)  # Adjust the input size based on the fft_length : (5000 : 623 | 10000 : 1248)
        self.fc2 = nn.Linear(256, 4)  # Output size is 4 for the number of music genres

    def forward(self, x):
        # Input shape: (batch_size, channels, sequence_length)
        #print("Step1 : " + str(x.shape))
        x = self.conv1(x)
        #print("Step2 : " + str(x.shape))
        x = F.relu(x)
        #print("Step3 : " + str(x.shape))
        x = self.pool(x)
        #print("Step4 : " + str(x.shape))
        x = self.conv2(x)
        #print("Step5 : " + str(x.shape))
        x = F.relu(x)
        #print("Step6 : " + str(x.shape))
        x = self.pool(x)
        #print("Step7 : " + str(x.shape))
        x = self.conv3(x)
        #print("Step8 : " + str(x.shape))
        x = F.relu(x)
        #print("Step9 : " + str(x.shape))
        x = self.pool(x)
        #print("Step10 : " + str(x.shape))
        x = self.conv4(x)
        #print("Step11 : " + str(x.shape))
        x = self.fc1(x)
        #print("Step12 : " + str(x.shape))
        x = F.relu(x)
        #print("Step13 : " + str(x.shape))
        x = self.fc2(x)
        #print("final : " + str(x.shape))
        return x.squeeze(1)

In [35]:
inputs = fourier_transforms
inputs_train, inputs_test, labels_train, labels_test = train_test_split(inputs, labels, test_size = 0.2, random_state=42)

model = MusicGenreCNN()
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learningRate)

#inputs_train = torch.tensor(inputs_train, dtype=torch.float32)
inputs_train = torch.stack(inputs_train).unsqueeze(1).float().to(device)
labels_train = torch.tensor(labels_train, dtype=torch.long).to(device)

train_dataset = TensorDataset(inputs_train, labels_train)
train_loader = DataLoader(train_dataset, batch_size = batchSize, shuffle=True)

for epoch in range(0, epochs):
    for inputs_batch, labels_batch in train_loader:
        optimizer.zero_grad()
        outputs_batch = model(inputs_batch)
        loss = criterion(outputs_batch, labels_batch)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

C:\Users\Tartiflettopolys\AppData\Local\Temp\ipykernel_15792\1903335218.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_train = torch.tensor(labels_train, dtype=torch.long).to(device)


Epoch 1, Loss: 1.2658710479736328
Epoch 2, Loss: 0.9966512322425842
Epoch 3, Loss: 0.7513201236724854
Epoch 4, Loss: 0.8038084506988525
Epoch 5, Loss: 0.7748227715492249
Epoch 6, Loss: 0.6800925731658936
Epoch 7, Loss: 0.6642637848854065
Epoch 8, Loss: 0.5072688460350037
Epoch 9, Loss: 0.532238245010376
Epoch 10, Loss: 0.5423945784568787
Epoch 11, Loss: 0.4160996377468109
Epoch 12, Loss: 0.2752603590488434
Epoch 13, Loss: 0.354959100484848
Epoch 14, Loss: 0.15416844189167023
Epoch 15, Loss: 0.2810055911540985
Epoch 16, Loss: 0.22488385438919067
Epoch 17, Loss: 0.21866804361343384
Epoch 18, Loss: 0.12036050111055374
Epoch 19, Loss: 0.1587965488433838
Epoch 20, Loss: 0.13570885360240936
Epoch 21, Loss: 0.08658400177955627
Epoch 22, Loss: 0.09542149305343628
Epoch 23, Loss: 0.10420740395784378
Epoch 24, Loss: 0.1034548357129097
Epoch 25, Loss: 0.06559410691261292
Epoch 26, Loss: 0.1279396414756775
Epoch 27, Loss: 0.09520822763442993
Epoch 28, Loss: 0.05744569003582001
Epoch 29, Loss: 0.16

### Tests

In [36]:
modelCPU = model.to('cpu')

inputs_test2 = torch.stack(inputs_test).unsqueeze(1).float()
labels_test2 = torch.tensor(labels_test, dtype=torch.long)

outputs_test = model(inputs_test2)
_, predicted = torch.max(outputs_test, 1)

correct = (predicted == labels_test2).sum().item()
accuracy = correct / labels_test2.size(0)
print(f'Accuracy of the network on test inputs: {accuracy * 100}%')

C:\Users\Tartiflettopolys\AppData\Local\Temp\ipykernel_15792\3277575847.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_test2 = torch.tensor(labels_test, dtype=torch.long)


Accuracy of the network on test inputs: 75.25%
